In [9]:
from IPython.display import display, Markdown, Code
import tempfile
from pathlib import Path, PurePath
import os
import subprocess
import shlex
import hashlib
import zlib

In [2]:
base = tempfile.mkdtemp(prefix='git-mock')

display(Code(f'cd {base}'))

`cd /tmp/git-mockj1my2to8`

In [3]:
init_empty_dirs = [
    'branches',
    'hooks',
    'info',
    'objects/info',
    'objects/pack',
    'refs/heads',
    'refs/tags'
]

git_dir = PurePath(base).joinpath('.git')
os.mkdir(git_dir)

for d in init_empty_dirs:
    os.makedirs(git_dir.joinpath(d), exist_ok=True)

In [4]:
with open(git_dir.joinpath('config'), 'wt', encoding='utf-8') as f:
    f.write('''\
[core]
    repositoryformatversion = 0
    filemode = true
    base = false
    logallrefupdates = true
''')

In [5]:
with open(git_dir.joinpath('HEAD'), 'wt', encoding='utf-8') as f:
    f.write('ref: refs/heads/master')

In [21]:
def run_cmd(cmd):
    proc = subprocess.run(shlex.split(cmd), capture_output=True, encoding='utf-8')

    display(Code(f'>>> {shlex.join(proc.args)}\n{proc.stdout}'))

In [22]:
run_cmd(f'git -C {base} status')

>>> git -C /tmp/git-mockj1my2to8 status
On branch master

No commits yet

nothing to commit (create/copy files and use "git add" to track)

In [12]:
def write_blob_object(file_content):
    raw_content = f'blob {len(file_content)}\x00{file_content}'.encode('utf-8')
    sha1 = hashlib.sha1(raw_content).hexdigest()
    
    compressed = zlib.compress(raw_content)
    object_dir = git_dir.joinpath('objects', sha1[:2])
    os.makedirs(object_dir, exist_ok=True)
    with open(object_dir.joinpath(sha1[2:]), 'wb') as f:
        f.write(compressed)
        
    return sha1

In [14]:
blob_sha = write_blob_object('very first file 1')
print(blob_sha)

2a94aaff68840af828318ce66927ef8782d9d5dd


In [23]:
run_cmd(f'git -C {base} cat-file -p {blob_sha}')

>>> git -C /tmp/git-mockj1my2to8 cat-file -p 2a94aaff68840af828318ce66927ef8782d9d5dd
very first file 1